HashMap: List of TODO things  
1) Constructur: Basic array with a initial number of elements pre-allocated.  
Note: this is not an array really, but a list  
2) hashing function:Returns the hash-index  
get_hash_code(), get_hash_index() : I've implemented as 2 different functions  
3) rehashing function  
4) search function  
5) put function: that takes care of collisions(use linked list where necessary)  
6) get function: that returns the value corresponding to key  
7) delete function  
8) print the hashmap function

In [1]:
class Node():
    def __init__(self, key=None, value=None, next_node=None):
        self.key   = key
        self.value = value
        self.next  = next_node


        
class HashMap():
    #Class construtor initialises the num_buckets (default is 10). Creates a list of objects initialised to None    
    def __init__(self, initial_num_buckets = 10):
        self.bucket_array = [None for _ in range(initial_num_buckets)]
        self.prime_coeff  = 31
        self.num_entries  = 0
        self.get          = self.search
    
    def get_load_factor(self):
        num_buckets= len(self.bucket_array)
        load_factor = self.num_entries/num_buckets
        return load_factor
    
        
    def get_hash_code(self, key):
        #convert the key to a string
        key       = str(key)
        hash_code = 0
        
        for char_count, character in enumerate(key):
            multiplicative_factor = pow(self.prime_coeff,char_count)
            hash_code += ord(character)*multiplicative_factor
        
        return hash_code
    
    def get_hash_index(self, key):
        num_buckets = len(self.bucket_array)
        hash_code = self.get_hash_code(key)
        
        #Compression factor: Divide the hash_code by the num_buckets and return the remainder
        hash_index = hash_code %num_buckets
        
        return hash_index
    
    def search(self, key):
        #obtain the hash_index of where the key-value pair will reside if it already exists
        bucket_arr_idx = self.get_hash_index(key)  
        #if the array index is empty, return search=False
        if(self.bucket_array[bucket_arr_idx] == None):
            return_index = [-1,-1]
            search_flag  = False
            value        = None
        #If array is not empty search through the open chain where the index is
        else:
            current = self.bucket_array[bucket_arr_idx]
            horizontal_chain_idx = 0
            search_flag = False
            while(current !=None):
                if(current.key == key):
                    search_flag  = True
                    return_index = [bucket_arr_idx,horizontal_chain_idx]
                    value        = current.value
                    break
                else:
                    horizontal_chain_idx +=1   
                    current = current.next
            
            #At the end of the while if key still not found assign return_idx =[-1,-1]
            if(search_flag == False):
                return_index = [-1,-1]
                value        = None
        
        return return_index,value,search_flag
            
            
    
    def put(self, key, value, allow_rehash = True):         
        #obtain the hash_index of where the key-value pair has to go
        arr_index = self.get_hash_index(key)            
        
        #if there are no other nodes at this arr_index, the new key-value becomes head
        if(self.bucket_array[arr_index] == None):
            new_node                     = Node(key, value)
            self.bucket_array[arr_index] = new_node
            self.num_entries             +=1
            put_success_flag             = True
        #if there are other nodes at this index. 
        else:
            search_index, search_value, key_exists = self.search(key)
            #Key already exists
            if(key_exists == True):
                print("Exiting without any insertions !!!. Key:", key ," already exists @index=", index)
                put_success_flag = False
            #Key does not exist. Insert it now as the new-head at the beginning of existing linked-list
            else:
                current_head                 = self.bucket_array[arr_index]
                new_node                     = Node(key, value, current_head)
                self.bucket_array[arr_index] = new_node
                self.num_entries +=1
                put_success_flag = True
        
        
        #The flag allw_rehash is necessary so that self.put and self.rehash are not in a race-condition/infinite loop
        if(put_success_flag == True and allow_rehash== True and self.get_load_factor() > 0.7):
            print("Added (key={}:value={}). Load factor exceeded ! Going to Rehash".format(key,value))
            print("HashMap Before Rehashing")
            print(self)
            
            self.rehash()
            
            print("HashMap After Rehashing")
            print(self)
        
        return put_success_flag
    
      
    def delete(self,key):        
        delete_index,value,search_flag = search(key)
        #Key does not exist
        if(search_flag == False):
            print("No Deletions: Key does not exist")
            delete_success_flag = False
        #Key exists
        else:
            bucket_idx           = delete_index[0]
            horizontal_chain_idx = delete_index[1]
            
            #Key to delete is at the head
            if(horizontal_chain_idx==0):
                key_2delete   = self.bucket_array[bucket_idx].key 
                value_2delete = self.bucket_array[bucket_idx].value
                self.bucket_array[bucket_idx].key   = None
                self.bucket_array[bucket_idx].value = None
                self.num_entries                    -= 1
                delete_success_flag                 = True
                print("Key:{}, Value:{}, deleted@index:{}".format(key2delete,value2delete,delete_index))
                
            #Key to delete is at the middle of the chain
            else:
                #Identify the previous_node of the node2_delete
                previous_node = self.bucket_array[bucket_idx]
                count   = 0
                while(count<=horizontal_chain_idx -1):
                    previous_node = previous_node.next
                    count+=1
                
                node2_delete  = previous_node.next
                key_2delete   = self.bucket_array[bucket_idx].key 
                value_2delete = self.bucket_array[bucket_idx].value
                
                next_node    = node2_delete.next
                #Link the previous_node(of node2_delete) and next_node(of node2_delete)
                previous_node.next = next_node
                #Actually delete node2_delete
                del node2_delete
                self.num_entries    -= 1
                delete_success_flag = True
                print("Key:{}, Value:{}, deleted@index:{}".format(key2delete,value2delete,delete_index))
        
        return delete_index, value, delete_success_flag
 

    def rehash(self):
        print("Rehashing !!! ")
        old_bucket_array  = self.bucket_array
        self.bucket_array = [None for _ in range(2*len(old_bucket_array))]
        self.num_entries  = 0
        for bucket_idx in range(len(old_bucket_array)):
            current_node = old_bucket_array[bucket_idx]
            while(current_node !=None):
                key   = current_node.key
                value = current_node.value
                #Now use the put function. 
                #Set allow_rehash = False,to avoid an infinite loop between the put-rehash methods
                self.put(key,value, allow_rehash = False)
                current_node = current_node.next
                
        return
            
    
    def __repr__(self):
        output = 'Below is HashMap:'
        for bucket_idx in range(len(self.bucket_array)):
            output +="\n[{}] ".format(bucket_idx)
            current = self.bucket_array[bucket_idx]
            while(current !=None):
                key    = current.key
                value  = current.value
                output += "({}:{}), ".format(key,value)
                current = current.next
                         
        output +="\n-----------------------------\n"
        
        return output
    
    
    
    
print("---------------------")

---------------------


In [2]:
#Test Hassmap with intial  7 entries
print("Compare with the code output of HashMap_Udacity to check correctness of results")
hash_map = HashMap(7)

hash_map.put("one", 1)
hash_map.put("two", 2)
hash_map.put("three", 3)
hash_map.put("neo", 11)

hash_map

Compare with the code output of HashMap_Udacity to check correctness of results


Below is HashMap:
[0] (three:3), 
[1] 
[2] (two:2), 
[3] 
[4] 
[5] (neo:11), 
[6] (one:1), 
-----------------------------

In [3]:
# Test HashMap with initial 5 entries. This should trigger rehashing automatically
print("Compare with the code output of HashMap_Udacity to check correctness of results")
hash_map = HashMap(5)

hash_map.put("one", 1)
hash_map.put("two", 2)
hash_map.put("three", 3)
hash_map.put("neo", 11)




Compare with the code output of HashMap_Udacity to check correctness of results
Added (key=neo:value=11). Load factor exceeded ! Going to Rehash
HashMap Before Rehashing
Below is HashMap:
[0] 
[1] (three:3), (two:2), 
[2] (neo:11), (one:1), 
[3] 
[4] 
-----------------------------

Rehashing !!! 
HashMap After Rehashing
Below is HashMap:
[0] 
[1] 
[2] (one:1), (neo:11), 
[3] 
[4] 
[5] 
[6] (two:2), (three:3), 
[7] 
[8] 
[9] 
-----------------------------



True